In [ ]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options

def fetch_kursusinformation_with_selenium(driver, semester, course_number):
    url = f"https://kurser.dtu.dk/course/{semester}/{course_number}"
    driver.get(url)
    
    time.sleep(2)  # Give it a moment to load fully if necessary
    
    # Get the page source and parse with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Find the specific "Kursusinformation" table or section within the div
    kursusinformation_section = soup.select_one("div.box.information")
    
    if kursusinformation_section:
        # Extract all the text within the Kursusinformation section
        kursusinformation_text = kursusinformation_section.get_text(separator="\n", strip=True)
        return kursusinformation_text
    else:
        return None  # Return None if no information is found

def save_kursusinformation_to_csv(semester, course_number, kursusinformation_text, filename="kursusinformation.csv"):
    # Write Kursusinformation data to a CSV file
    with open(filename, mode='a', newline='', encoding='utf-8') as file:  # Use 'a' mode to append to the file
        writer = csv.writer(file)
        
        # Write the header only if the file is empty
        if file.tell() == 0:
            writer.writerow(["Semester", "Course Number", "Kursusinformation"])
        
        # Write the Kursusinformation with the semester and course number
        writer.writerow([semester, course_number, kursusinformation_text])

# List of semesters from 2017-2018 to 2023-2024
semesters = [f"{year}-{year+1}" for year in range(2017, 2024)]

# Generate the first 20 course numbers between 01000 to 88718
course_numbers = [str(num).zfill(5) for num in range(1000, 88718)]  # Adjust this range as needed

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Optional: Start maximized
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model

# Start the browser session once
driver = webdriver.Chrome(options=chrome_options)

try:
    # Loop through each semester and the first 20 course numbers and scrape data
    for semester in semesters:
        for course_number in course_numbers:
            kursusinformation_text = fetch_kursusinformation_with_selenium(driver, semester, course_number)
            if kursusinformation_text:
                print(f"Fetched Kursusinformation for {semester} - {course_number}")
                # Save the extracted Kursusinformation to a CSV file
                save_kursusinformation_to_csv(semester, course_number, kursusinformation_text, filename="kursusinformation.csv")
finally:
    # Close the browser session when done
    driver.quit()

print("Data fetching complete. Results saved to kursusinformation.csv")